In [ ]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

# ==========================================================
# 1. 加载模型
# ==========================================================
model_name = "models/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to("mps")
model.gradient_checkpointing_enable()
print(model.is_gradient_checkpointing)
print(model.device)

# ==========================================================
# 2. LoRA 配置
# ==========================================================
lora_config = LoraConfig(
    r=8, lora_alpha=16, target_modules=["q_proj","v_proj"],
    lora_dropout=0.1, bias="none", task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
print("模型参数量（含LoRA）:", model.get_memory_footprint())
# 模型参数量（含LoRA）: 1978293888
# 模型参数量（含LoRA）: 990228352


/Users/bierxiensi/Desktop/paper/flood-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bierxiensi/Desktop/paper/flood-rag/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


True
mps:0
模型参数量（含LoRA）: 990228352


In [2]:
from transformers import DataCollatorForLanguageModeling

# ==========================================================
# 3. 加载数据集（JSONL）
# ==========================================================
dataset = load_dataset("json", data_files={"train": "./data/train.jsonl", "test": "./data/test.jsonl"})

# ==========================================================
# 4. 数据预处理函数
# ==========================================================
def format_example(example):
    # 拼接 instruction + input + output 为单一文本（SFT格式）
    text = f"Instruction: {example['instruction']}\nInput: {example['input']}\nAnswer: {example['output']}"
    return {"text": text}

dataset = dataset.map(format_example)

def tokenize_fn(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=1024,
        padding="max_length"
    )

tokenized_datasets = dataset.map(tokenize_fn, batched=True, remove_columns=dataset["train"].column_names)

# ==========================================================
# 5. 数据整理器
# ==========================================================
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [3]:
from transformers import TrainingArguments, Trainer
# 4. 训练参数
training_args = TrainingArguments(
    output_dir="./rag_lora",
    per_device_train_batch_size=1,
    learning_rate=2e-4,
    gradient_accumulation_steps=4,  # 累积梯度模拟更大 batch
    num_train_epochs=3,
    save_strategy="epoch",
    logging_steps=50,
    optim="adamw_torch",
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/var/folders/f_/vl89sy093p90zv2lhwbwl92w0000gn/T/ipykernel_26047/2894822238.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/Users/bierxiensi/Desktop/paper/flood-rag/.venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss


KeyboardInterrupt: 